<a href="https://colab.research.google.com/github/kevinhhl/portfolio-management-tools/blob/main/Correlation_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Manual inputs
TIMEFRAME = "1d"
LOOKBACK_PERIOD = "5y" 
UNIVERSE = ["QQQ", "SPY", "TSLA", "AAPL", "TSM", "MRK", "KO", "MCD", "MSFT", "JNJ", "PFE", "PM"]

#Implementation:

In [ ]:
!pip install yfinance
import pandas as pd
import yfinance as yf
from datetime import datetime
import numpy as np

In [3]:
stock_list = []
for stock in UNIVERSE:
  returns = yf.Ticker(stock).history(period=LOOKBACK_PERIOD, interval=TIMEFRAME)
  returns["returns"] =  np.log(1+returns["Close"].pct_change()) # returns["Close"].pct_change()
  returns.rename(columns={"returns": stock}, inplace=True)
  stock_list.append(returns[stock])

In [4]:
correl_matrix = pd.DataFrame(stock_list).T.corr().round(4)

#Results:

In [5]:
correl_matrix.style.background_gradient(cmap ='coolwarm')

,QQQ,SPY,TSLA,AAPL,TSM,MRK,KO,MCD,MSFT,JNJ,PFE,PM
QQQ,1.000000,0.934100,0.570800,0.864300,0.683200,0.404900,0.481500,0.526700,0.907600,0.464400,0.402200,0.395600
SPY,0.934100,1.000000,0.486300,0.803700,0.644900,0.512700,0.645900,0.651500,0.849800,0.579100,0.497400,0.535700
TSLA,0.570800,0.486300,1.000000,0.477200,0.413100,0.133600,0.196500,0.291900,0.465800,0.131800,0.102800,0.167300
AAPL,0.864300,0.803700,0.477200,1.000000,0.583300,0.346100,0.421400,0.460600,0.769500,0.412600,0.345800,0.361700
TSM,0.683200,0.644900,0.413100,0.583300,1.000000,0.233800,0.275400,0.361300,0.574500,0.268100,0.190600,0.326800
MRK,0.404900,0.512700,0.133600,0.346100,0.233800,1.000000,0.470300,0.444500,0.411700,0.578000,0.511800,0.411500
KO,0.481500,0.645900,0.196500,0.421400,0.275400,0.470300,1.000000,0.554500,0.456200,0.558700,0.445600,0.546200
MCD,0.526700,0.651500,0.291900,0.460600,0.361300,0.444500,0.554500,1.000000,0.499400,0.438100,0.314600,0.471800
MSFT,0.907600,0.849800,0.465800,0.769500,0.574500,0.411700,0.456200,0.499400,1.000000,0.455800,0.387800,0.361100
JNJ,0.464400,0.579100,0.131800,0.412600,0.268100,0.578000,0.558700,0.438100,0.455800,1.000000,0.551700,0.445800
